In [2]:
from keras.applications import Xception
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model

base_model = Xception(False, "imagenet")
for layer in base_model.layers:
    layer.trainable = True
    
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)
model = Model(base_model.input, predictions)

In [3]:
from Datasets import Datasets

dataset_name = "Bradbury"
dataset = Datasets.datasets()[dataset_name]

In [4]:
from keras.applications.xception import preprocess_input
import numpy as np

# Add preprocessing
train_images = dataset[0].images
train_labels  = dataset[0].labels

test_images = dataset[1].images
test_labels = dataset[1].labels

validation_images = dataset[2].images
validation_labels = dataset[2].labels


In [5]:
validation_images.shape
validation_labels.shape

(3973,)

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt


In [7]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()


In [8]:
from PerformanceMetrics import PerformanceMetrics
from keras.optimizers import SGD

model.compile(optimizer="rmsprop", loss='binary_crossentropy',
                  metrics=['accuracy', PerformanceMetrics.precision,
                           PerformanceMetrics.recall, PerformanceMetrics.fmeasure])

In [10]:
batch_size = 64

train_generator = data_generator.flow(train_images, train_labels, batch_size=batch_size)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=batch_size, shuffle=False)

In [11]:
import os
from ProjectPaths import ProjectPaths
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping


epochs = 100

model_name = "xception_idg_{}".format(dataset_name)

checkpoint_dir = ProjectPaths.checkpoint_dir_for(model_name, batch_size, epochs)
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)

file_in_checkpoint_dir = ProjectPaths.file_in_checkpoint_dir(model_name, batch_size,
                                                                 epochs,  model_name +
                                                                 "__{epoch:02d}_{val_acc:.2f}.hdf5")

early_stopping_callback = EarlyStopping(patience=10)
model_checkpoint_callback = ModelCheckpoint(file_in_checkpoint_dir, monitor='val_acc', verbose=True,
                                                save_weights_only=True,
                                                save_best_only=True)

log_dir = os.path.join(ProjectPaths.log_dir(), model_name)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=0,  write_graph=False, write_images=False)


model.fit_generator(train_generator,
                    steps_per_epoch=len(train_labels) // batch_size, 
                    epochs=epochs,
                    callbacks=[early_stopping_callback, model_checkpoint_callback, tensorboard_callback],
                    validation_data=test_generator,
                    validation_steps=len(test_labels))



Epoch 1/100
496/496 [==============================] - 57s 114ms/step - loss: 0.2943 - acc: 0.8409 - precision: 0.8147 - recall: 0.8562 - fmeasure: 0.8178 - val_loss: 0.3780 - val_acc: 0.9094 - val_precision: 0.8497 - val_recall: 0.9930 - val_fmeasure: 0.9145

Epoch 00001: val_acc improved from -inf to 0.90939, saving model to /media/tim/Data/Work/CBS/Code_hannah/Models/xception_idg_Bradbury_2018-08-31_64_100/xception_idg_Bradbury__01_0.91.hdf5
Epoch 2/100
496/496 [==============================] - 54s 108ms/step - loss: 0.1121 - acc: 0.9573 - precision: 0.9589 - recall: 0.9569 - fmeasure: 0.9567 - val_loss: 0.1261 - val_acc: 0.9643 - val_precision: 0.9510 - val_recall: 0.9775 - val_fmeasure: 0.9635

Epoch 00002: val_acc improved from 0.90939 to 0.96426, saving model to /media/tim/Data/Work/CBS/Code_hannah/Models/xception_idg_Bradbury_2018-08-31_64_100/xception_idg_Bradbury__02_0.96.hdf5
Epoch 3/100
496/496 [==============================] - 54s 109ms/step - loss: 0.0933 - acc: 0.9659 


Epoch 00024: val_acc did not improve from 0.98213
Epoch 25/100
496/496 [==============================] - 54s 108ms/step - loss: 0.0408 - acc: 0.9872 - precision: 0.9897 - recall: 0.9849 - fmeasure: 0.9870 - val_loss: 0.0745 - val_acc: 0.9829 - val_precision: 0.9778 - val_recall: 0.9868 - val_fmeasure: 0.9820

Epoch 00025: val_acc improved from 0.98213 to 0.98288, saving model to /media/tim/Data/Work/CBS/Code_hannah/Models/xception_idg_Bradbury_2018-08-31_64_100/xception_idg_Bradbury__25_0.98.hdf5


# Evaluation of Bradbury

In [13]:
import numpy as np
import pandas as pd

train_eval = model.evaluate(train_images, train_labels, batch_size)
test_eval = model.evaluate(test_images, test_labels, batch_size)
validation_eval = model.evaluate(validation_images, validation_labels, batch_size)

np_model_evaluations = np.array([train_eval, test_eval, validation_eval])

evaluations = pd.DataFrame(np_model_evaluations, columns=model.metrics_names)
print(evaluations.head())

3973/3973 [==============================] - 2s 432us/step
       loss       acc  precision    recall  fmeasure
0  0.036073  0.989520   0.989427  0.989660  0.989373
1  0.074477  0.982884   0.977841  0.986821  0.981953
2  0.086786  0.978102   0.975374  0.981704  0.978224


In [27]:
from sklearn.metrics import confusion_matrix

validation_predictions = model.predict(validation_images, batch_size) > 0.5
confusion_matrix(validation_labels, validation_predictions)



array([[412,  45],
       [154, 323]])

In [31]:
# What happened to the metrics above?? Bug in Hannah's metrics?
print(classification_report(validation_labels, validation_predictions))

             precision    recall  f1-score   support

        0.0       0.73      0.90      0.81       457
        1.0       0.88      0.68      0.76       477

avg / total       0.80      0.79      0.78       934



# Evaluation of Aachen

Evaluation of the Aachen dataset with the model trained on Bradbury California

In [24]:
ac_dataset = Datasets.datasets()["AcMüDüHo"]

train_images = ac_dataset[0].images
train_labels = ac_dataset[0].labels

test_images = ac_dataset[1].images
test_labels = ac_dataset[1].labels

validation_images = ac_dataset[2].images
validation_labels = ac_dataset[2].labels

eval_images = np.concatenate((train_images, test_images, validation_images), axis=0)
eval_labels = np.concatenate((train_labels, test_labels, validation_labels), axis=0)


print(train_images.shape, test_images.shape, validation_images.shape)
print(eval_images.shape, eval_labels.shape)

(3357, 75, 75, 3) (374, 75, 75, 3) (934, 75, 75, 3)
(4665, 75, 75, 3) (4665,)


In [28]:
eval_predictions = model.predict(eval_images, batch_size) > 0.5
confusion_matrix(eval_labels, eval_predictions)

array([[2101,  233],
       [ 822, 1509]])

In [30]:
from sklearn.metrics import classification_report

print(classification_report(eval_labels, eval_predictions))

             precision    recall  f1-score   support

        0.0       0.72      0.90      0.80      2334
        1.0       0.87      0.65      0.74      2331

avg / total       0.79      0.77      0.77      4665



# Evaluation of Fresno

In [33]:
fresno_dataset = Datasets.datasets()["Fresno"]

train_images = fresno_dataset[0].images
train_labels = fresno_dataset[0].labels

test_images = fresno_dataset[1].images
test_labels = fresno_dataset[1].labels

validation_images = fresno_dataset[2].images
validation_labels = fresno_dataset[2].labels

fresno_eval_images = np.concatenate((train_images, test_images, validation_images), axis=0)
fresno_eval_labels = np.concatenate((train_labels, test_labels, validation_labels), axis=0)


print(train_images.shape, test_images.shape, validation_images.shape)
print(fresno_eval_images.shape, fresno_eval_labels.shape)

(3646, 75, 75, 3) (1022, 75, 75, 3) (1022, 75, 75, 3)
(5690, 75, 75, 3) (5690,)


In [34]:
fresno_eval_predictions = model.predict(fresno_eval_images, batch_size) > 0.5
confusion_matrix(fresno_eval_labels, fresno_eval_predictions)

array([[2809,   36],
       [  19, 2826]])

In [35]:
from sklearn.metrics import classification_report

print(classification_report(fresno_eval_labels, fresno_eval_predictions))

             precision    recall  f1-score   support

          0       0.99      0.99      0.99      2845
          1       0.99      0.99      0.99      2845

avg / total       0.99      0.99      0.99      5690

